# Creating two unified databases, with only the necessary information

We are going to create a database with the largest train stations in each country of interest. 

These countries are France, Belgium, Switzerland, Germany and Austria. Furthermore, we can look at the trains in other parts of Europe aswell

We'll use a webscraping method to collect the information on a country, and save the list of cities we'll consider in a dictionary

In [ ]:
!pip install openpyxl
!pip install pandas fiona shapely pyproj rtree 
!pip install geopandas
!pip install folium

"Libraries Import"
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import pandas as pd
import geopandas as gpd
import countriesAndCities
import dataGathering

We will create a dictionary for Austria and Germany, containing each countries' largest train stations, based on these two links

In [ ]:
largestStations = dict()

urlGermany = 'https://bahnauskunft.info/bahnhoefe-deutschland/'
urlAustria = 'https://www.omio.at/bahnhoefe'

At the same time, we will have to change certain dictionary keys. As such, we can create a function that does exactly this.

In [ ]:
def changeKeys(country, valueToChange, newValue):
    '''A method that takes the keys for a country in the largestStations dictionary, and replacey certain values
     @param country: the country with a value to change, of type string
     @param valueToChange: the value in the key to change
     @param newValue: the new value in the key
     @return largestStations: a dictionary with the information, of type dict'''
    listKeys = list(largestStations[country].keys())
    oldKeys = []
    for i in range (len(listKeys)):
        station = listKeys[i]
        if valueToChange in listKeys[i]:
            oldKey = station
            oldKeys.append(oldKey)
            newKey = station.replace(valueToChange, '') + newValue
            largestStations[country][newKey] = largestStations[country][oldKey]
    
    for station in oldKeys:
        largestStations[country].pop(station)
    
    return(largestStations)

# 1. Germany

We'll start by getting the different relevant cities for Germany, and then working on the different geojson files to create a relevant database

In [ ]:
largestStations['Germany'] = dataGathering.gather(urlGermany)

In [ ]:
largestStations

{'Germany': {'Hamburg Hauptbahnhof': ['537.000', '12'],
  'Frankfurt (Main) Hauptbahnhof': ['493.000', '29'],
  'München Hauptbahnhof': ['413.000', '34'],
  'Berlin Hauptbahnhof': ['329.000', '14'],
  'Köln Hauptbahnhof': ['318.000', '11'],
  'Hannover Hauptbahnhof': ['261.000', '12'],
  'Stuttgart Hauptbahnhof': ['255.000', '19'],
  'Düsseldorf Hauptbahnhof': ['246.000', '16'],
  'Nürnberg Hauptbahnhof': ['210.000', '22'],
  'Essen Hauptbahnhof': ['152.000', '13'],
  'Bremen Hauptbahnhof': ['147.000', '9'],
  'Duisburg Hauptbahnhof': ['130.000', '12'],
  'Leipzig Hauptbahnhof': ['135.000', '23'],
  'Dortmund Hauptbahnhof': ['123.000', '16'],
  'Mannheim Hauptbahnhof': ['118.000', '11']}}

The deutsche Bahn's database uses 'Hbf' instead of 'Hauptbahnhof' so we must change the key

In [ ]:
largestStations = changeKeys('Germany', 'Hauptbahnhof', 'Hbf')

# 1.1. Stations

We'll start by creating a database of all stations in the selected cities

In [ ]:
stations = 'Germany/railwayStationNodes.geojson'

deutscheBahnStations = gpd.read_file(stations)

In [ ]:
deutscheBahnStations.columns

We can start by dropping the column containing the nature of the node, and the index of the node

In [ ]:
deutscheBahnStations = deutscheBahnStations.drop('formOfNode', axis = 1)
deutscheBahnStations = deutscheBahnStations.drop('id', axis = 1)

In [ ]:
deutscheBahnStations

We check, for every single row, if the name of the station is located in one of the select few cities. The name of the station is the third value (index 2) of each row
Furthermore, we check that there is space after the name of each city, as to avoid other cities with street names (as in Berlin -> Berliner).

We are going to select every line containing data on a station in one of the cities, and concatenate every one of these separate dataframe

In [ ]:
dfListStations = []
for station in (list(largestStations['Germany'].keys())):
    tempFrame = deutscheBahnStations.loc[deutscheBahnStations['geographicalName'] == station]
    dfListStations.append(tempFrame)

We define the geodataframe with the chosen coordinate system, EPSG:4258 (documentation available at https://www.geoportal.de/Metadata/55134453-193d-47ea-9b20-0f7016702c91, in german)

In [ ]:
workFrameStations = gpd.GeoDataFrame(pd.concat(dfListStations, ignore_index=True), crs=4258)

In [ ]:
workFrameStations

Certain nodes are the same station. We will keep a single occurence of every station, based on the railwayStationCode variable

In [ ]:
workFrameStations = workFrameStations.drop_duplicates(subset='railwayStationCode')

In [ ]:
workFrameStations

Finally, we can add a column, indicating that every value in this geodataframe is located in Germany

In [ ]:
workFrameStations.insert(0, 'country', ['Germany']*len(workFrameStations))

In [ ]:
workFrameStations

# 1.2. Lines

We can do the exact same thing the dataframe of the different train lines

In [ ]:
deutscheBahnLines = gpd.read_file('Germany/railwayLines.geojson')

In [ ]:
deutscheBahnLines

We can drop any railway line that doesn't start or end in one of our selected cities, i.e. create a new dataframe, with the lines that start or end in one of these cities

In [ ]:
dfListLines = []
for station in (list(largestStations['Germany'].keys())):
    tempFrameLines = deutscheBahnLines.loc[deutscheBahnLines['geographicalName'].str.contains(city + ' ')]
    dfListLines.append(tempFrameLines)

In [ ]:
workFrameLines = gpd.GeoDataFrame(pd.concat(dfListLines, ignore_index = True), crs = 4258)

In [ ]:
workFrameLines

And drop the duplicate lines

In [ ]:
workFrameLines = workFrameLines.drop_duplicates(subset='railwayLineCode')

In [ ]:
workFrameLines

Furthermore, we can drop the line id column

In [ ]:
workFrameLines = workFrameLines.drop('id', axis=1)

In [ ]:
workFrameLines

Add a column indicating that these lines are located in Germany

In [ ]:
workFrameLines.insert(0, 'country', ['Germany']*len(workFrameLines))

In [ ]:
workFrameLines

# 2. Austria

We now have a database with the different stations and lines in Germany. We will now add the values for Austria to this database.

In [ ]:
largestStations['Austria'] = dataGathering.gather(urlAustria, start = 1)

In [ ]:
largestStations

{'Germany': {'Hamburg Hbf': ['537.000', '12'],
  'Frankfurt (Main) Hbf': ['493.000', '29'],
  'München Hbf': ['413.000', '34'],
  'Berlin Hbf': ['329.000', '14'],
  'Köln Hbf': ['318.000', '11'],
  'Hannover Hbf': ['261.000', '12'],
  'Stuttgart Hbf': ['255.000', '19'],
  'Düsseldorf Hbf': ['246.000', '16'],
  'Nürnberg Hbf': ['210.000', '22'],
  'Essen Hbf': ['152.000', '13'],
  'Bremen Hbf': ['147.000', '9'],
  'Duisburg Hbf': ['130.000', '12'],
  'Leipzig Hbf': ['135.000', '23'],
  'Dortmund Hbf': ['123.000', '16'],
  'Mannheim Hbf': ['118.000', '11']},
 'Austria': {'Wien Hbf': ['268.000', '16'],
  'Wien Westbahnhof': ['43.000', '11'],
  'Innsbruck Hbf': ['34.000', '11'],
  'Linz Hbf': ['30.000', '14'],
  'Graz Hbf': ['30.000', '11'],
  'Salzburg Hbf': ['25.000', '10']}}

This database does the exact opposite of the German database. It uses 'Hauptbahnhof', whereas we had values with 'Hbf'

In [ ]:
largestStations = changeKeys('Austria', 'Hbf', 'Hauptbahnhof')

# 2.1. Stations

In [ ]:
stationsAustria = 'Austria/GIP_Betriebsstellen_DelEUV_JSON.json'
stationsAustriaFrame = gpd.read_file(stationsAustria)

In [ ]:
stationsAustriaFrame

,BSTS_ID,DB640_CODE,OBJECTID,GIP_OBID,EXTERNALID,REGIONALCO,VALIDFROM,VALIDTO,OWNER_NAME,NAME_FPL,...,MUEZ_KURZ,HILFE_MOBI,ANZ_ROLLST,ANZ_E_LADE,RUD_PARKPL,VERIFIZIER,PUBL_WLAN,MUEZ_LANG,BEMERKUNG,geometry
0,2082,Ssl,32,15532449605,bstsid:2082,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,St. Stefan im Lavanttal,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (14.84242 46.80693)
1,64,An,33,15532449608,bstsid:64,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,St. Andrä im Lavanttal,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (14.83744 46.76271)
2,403,Fas,34,15532449611,bstsid:403,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,Faak am See,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (13.90901 46.56449)
3,2337,Vu H1,37,15532449620,bstsid:2337,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,Gödersdorf,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (13.84352 46.56849)
4,2338,Vu H2,38,15532449623,bstsid:2338,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,Finkenstein,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (13.86439 46.56875)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1074,Ko H2,1088,15532741643,bstsid:1074,AT3,20201213000000,20211211000000,ÖBB Infrastruktur,Spillern,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (16.25258 48.37603)
1026,5267955,Mun H1,1089,15532741646,bstsid:5267955,AT4,20201213000000,20211211000000,ÖBB Infrastruktur,Schalchen-Mattighofen,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (13.15906 48.09566)
1027,5275915,Ob H1H,1090,15532741649,bstsid:5275915,AT4,20201213000000,20211211000000,ÖBB Infrastruktur,Oberland,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (14.74011 47.90194)
1028,5091980,Kts,1091,15532741652,bstsid:5091980,AT2,20201213000000,20211211000000,ÖBB Infrastruktur,Klagenfurt Süd,...,3,Nein,0,0,Nein,Nein,Nein,None,None,POINT (14.29839 46.61078)


In [ ]:
stationsAustriaFrame.columns

Index(['BSTS_ID', 'DB640_CODE', 'OBJECTID', 'GIP_OBID', 'EXTERNALID',
       'REGIONALCO', 'VALIDFROM', 'VALIDTO', 'OWNER_NAME', 'NAME_FPL',
       'IFOPT_NR', 'PV_EVA_NR', 'ANZ_AUFZUG', 'ANZ_FAHRTR', 'ANZ_UHREN',
       'ANZ_AKUSTI', 'ANZ_OPTISC', 'INFOPOINT', 'MUEZ', 'MUEZ_KURZ',
       'HILFE_MOBI', 'ANZ_ROLLST', 'ANZ_E_LADE', 'RUD_PARKPL', 'VERIFIZIER',
       'PUBL_WLAN', 'MUEZ_LANG', 'BEMERKUNG', 'geometry'],
      dtype='object')

Quite a few columns here are useless. We can remove these columns

In [ ]:
columnsToRemove = ['BSTS_ID', 'DB640_CODE', 'OBJECTID', 'GIP_OBID', 'EXTERNALID', 'REGIONALCO', 'VALIDFROM', 'VALIDTO', 'OWNER_NAME', 'PV_EVA_NR', 'ANZ_AUFZUG', 'ANZ_FAHRTR', 'ANZ_UHREN',
                  'ANZ_AKUSTI','ANZ_OPTISC', 'INFOPOINT', 'MUEZ', 'MUEZ_KURZ', 'HILFE_MOBI', 'ANZ_ROLLST', 'ANZ_E_LADE', 'RUD_PARKPL', 'VERIFIZIER',
                  'PUBL_WLAN', 'MUEZ_LANG', 'BEMERKUNG']

In [ ]:
for column in columnsToRemove:
    stationsAustriaFrame = stationsAustriaFrame.drop(column, axis=1)

In [ ]:
stationsAustriaFrame

,NAME_FPL,IFOPT_NR,geometry
0,St. Stefan im Lavanttal,at:42:3717,POINT (14.84242 46.80693)
1,St. Andrä im Lavanttal,at:42:3718,POINT (14.83744 46.76271)
2,Faak am See,at:42:2139,POINT (13.90901 46.56449)
3,Gödersdorf,at:42:2137,POINT (13.84352 46.56849)
4,Finkenstein,at:42:2138,POINT (13.86439 46.56875)
...,...,...,...
1025,Spillern,at:43:4814,POINT (16.25258 48.37603)
1026,Schalchen-Mattighofen,at:44:48315,POINT (13.15906 48.09566)
1027,Oberland,at:44:46900,POINT (14.74011 47.90194)
1028,Klagenfurt Süd,at:42:3842,POINT (14.29839 46.61078)


We can now focus on retrieving the rows with information on the two cities of interest

In [ ]:
dfStationsAustria = []
for station in largestStations['Austria']:
    tempFrame = stationsAustriaFrame.loc[stationsAustriaFrame['NAME_FPL'] == station]
    dfStationsAustria.append(tempFrame)

KeyError: 'Austria'

In [ ]:
dfStationsAustria

In [ ]:
workFrameAustria = gpd.GeoDataFrame(pd.concat(dfStationsAustria), crs = 31287)

In [ ]:
workFrameAustria

Add the country to the work dataframe

In [ ]:
workFrameAustria.insert(0, 'country', ['Austria']*len(workFrameAustria))

In [ ]:
workFrameAustria

# 2.2. Lines

We can do the exact same with the train lines database

In [ ]:
linesAustria = 'Austria/GIP_Strecken_MLA.json'
linesAustriaFrame = gpd.read_file(linesAustria)

In [ ]:
linesAustriaFrame

In [ ]:
linesAustriaFrame.columns

Once again, quite a few columns are useless, and we can get rid of them

In [ ]:
uselessColumns = ['GIP_OBID', 'BST_ID', 'FOW_NAME', 'FRC_NAME', 'REGION', 'VALIDFROM', 'VALIDTO', 'CROSSSECT', 'CROSS_NAME', 
                  'ELEKTRI', 'EXPDATE']

for column in uselessColumns:
    linesAustriaFrame = linesAustriaFrame.drop(column, axis=1)

In [ ]:
linesAustriaFrame

# 3. France

We are using databases coming from French government open-data portal and French national railway company (SNCF) : https://www.data.gouv.fr/fr/ and https://ressources.data.sncf.com/pages/accueil/ \
We focus on two databases providing information either on the railway stations or on tne rail connections. \
The databases are : referentiel-gares-voyageurs.geojson and formes-des-lignes-du-rfn.geojson

# 3.1. Stations

We first select the relevant cities. We focus on a dozen of them, the biggest ones. We chose : Paris, Lyon, Bordeaux, Toulouse, Brest, Marseille, Nice, Strasbourg, Rennes, Motpellier and Grenoble. \
Some of these cities have several stations. For more simplicity, there is a focus on the relevant stations connecting those cities.

In [ ]:
f_station = gpd.read_file('France/referentiel-gares-voyageurs.geojson')

In [ ]:
values = ['Paris', 'Lyon', 'Strasbourg', 'Lille', 'Bordeaux', 'Massy', 'Marseille', 'Chessy', 'Brest', 'Rennes', 'Toulouse', 'Nice', 'Montpellier', 'Grenoble']
f_station = f_station[f_station.commune_libellemin.isin(values)] 

f_station = f_station[f_station['segmentdrg_libelle']=='a']	

There is a selection on two criteria : the name of the city and the importance of the line between cities (segmentdrg_libelle). \
There is a focus on national and international railways. \


Note : To achieve our aim we use the two columns called commune_libellemin and segmentdrg_libelle. \
commune_libellemin = name of the city \
segmentdrg_libelle = a,b or c indicates the size of the railroad

In [ ]:
f_station.loc[:,'Country'] = 'France'

We also have an additional column giving the name of the country and we drop out the contingent columns.

In [ ]:
f_station.drop(['tvs', 'tvss',
       'gare_alias_libelle_fronton', 'commune_code', 'uic_code', 'rg_libelle',
       'gare', 'adresse_cp', 'code_gare', 'latitude_entreeprincipale_wgs84',
       'gare_agencegc_libelle', 'departement_libellemin', 'code',
       'gare_drg_on', 'gare_etrangere_on', 'segmentdrg_libelle',
       'gare_regionsncf_libelle', 'longitude_entreeprincipale_wgs84',
       'gare_ut_libelle', 'gare_nbpltf', 'alias_libelle_noncontraint',
       'departement_numero', 'niveauservice_libelle', 'gare_ug_libelle'],1,inplace=True)

# 3.2. Lines

Now, we select the relevant lines between those cities

In [ ]:
lines_france = gpd.read_file('France/formes-des-lignes-du-rfn.geojson')

We only focus on the exploited lines. We drop the lines that are unexploited or private.

In [ ]:
lines_france.drop(lines_france.loc[lines_france['libelle'] != 'Exploitée'].index, inplace = True)

## Selection the id of the lines of interest

The interesting lines have special ids. We collected them in order to plot only the most interesting lines. We chose to focus on the fastest lines and dropped all the duplicates. For example, between Lyon and Paris, we only add the high-speed line connecting both cities. We do not add the normal-speed line which serves minor cities such as Auxerre or Dijon. \
The ids allow us to select the relevant lines :

In [ ]:
values = [ '930000', '752000', '810000', '640000', '566000','956000', '957000', '991300', '752330', '893000', '431000', '420000','834000', '226000', '005000', '905000', '752340', '226310', '005390', '752100' ]

In [ ]:
lines_france = lines_france[lines_france.code_ligne.isin(values)] 

# 4. Belgium

# 4.1. Stations

Because of the small size of the country, there is a focus on the cities of Brussels, Antwerp, and Gent. We also add the connection with France : line between Brussels and Lille in France, and between Antwerp, Rotterdam and Amsterdam in the Netherlands. \
Because of the poor data , we chose to create our own dataset. The very small amount of datas studied is a key element of such a choice. 
So, there is the creation of a dataframe with relevant coordinates.

In [ ]:
df = pd.DataFrame(
    {'City' : ['Brussel', 'Brussel','Gent', 'Atwerpen', 'Rotterdam',"Amsterdam"],
     'Name': ['Brussel-Zuid','Brussel-Noord','Gent-Sint-Pieters','Antwerpen-Centraal','Rotterdam-Centraal', 'Amsterdam-Centraal'],
     'Country': ['Belgium', 'Belgium', 'Belgium', 'Belgium', 'Netherland', 'Netherland'],
     'Longitude': [4.33545, 4.36015, 3.71083, 4.42115, 4.46883, 4.90032],
     'Latitude': [50.83605, 50.86045, 51.03609, 51.21722, 51.92528, 52.37913]})

Then, transformation of the dataframe to a geopandas one : the coordinates are transformed into a geometry recognized when plotted on a map.

In [ ]:
b_station = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [ ]:
b_station.drop(['Longitude', 'Latitude'],1, inplace = True)

,City,Name,Country,geometry
0,Brussel,Brussel-Zuid,Belgium,POINT (4.33545 50.83605)
1,Brussel,Brussel-Noord,Belgium,POINT (4.36015 50.86045)
2,Gent,Gent-Sint-Pieters,Belgium,POINT (3.71083 51.03609)
3,Atwerpen,Antwerpen-Centraal,Belgium,POINT (4.42115 51.21722)
4,Rotterdam,Rotterdam-Centraal,Netherland,POINT (4.46883 51.92528)


# 4.2. Lines

As for the French lines, I focused on the lines'id to select the most accurate ones. The database is not very well organised because some of the lines are not continuous. 

In [ ]:
lines_belgium = gpd.read_file('Belgium/tc-trajet-train-statique-sncb.geojson')

In [ ]:
values = [603,140,166,126,259,257,54,274,172,602]
lines_belgium = lines_belgium[lines_belgium.id.isin(values)] 
lines_belgium.index = range(10)

Here, I add a columns of the departure city and country and the arrival one. These are the last changes of the database.

In [ ]:
departure = []
arrival = []
for k in range(10) :
  index_ = lines_belgium.long_name[k].index('--')
  departure.append(lines_belgium["long_name"][k][:index_])
  arrival.append(lines_belgium["long_name"][k][index_+3:])

lines_belgium.loc[:,"Departure"] = departure
lines_belgium.loc[:,"Arrival"] = arrival
lines_belgium.loc[:,"Country_dep"] = ['France', 'Belgium', 'Belgium', 'Belgium', 'Belgium', 'France', 'Netherland','Belgium', 'Belgium', 'Netherland']
lines_belgium.loc[:,"Country_arr"] = 'Belgium'
lines_belgium.drop(['short_name', 'long_name', 'id', 'route_type', 'color'],1,inplace=True)

# 5. Swtitzerland

# 5.1. Stations

I use the BAV_List_future_timetable.xlsx from https://opentransportdata.swiss/fr/dataset/bav_liste that I named suissedata1.xlsx


In [ ]:

df=pd.read_excel('suissedata1.xlsx')
df=df.drop(columns=['Remarque','Statut','Localité','N° commune','Ct.','Carte','Carte.1','N° sv.85','py','N° sv.','Cc','PE','PT','N° ET','Sigle ET','N° GO','Sigle GO','Nom long','Sigle sv.','PC','PP','ST'])
df.head()


#We only use the dataframe where the transport is equal to 'Zug' (Train)

df1=df.copy()

df1=df1[df1['Moyen de transport']=='Zug']
df1.head()


df2=df1[df1['Longueur']>20]
df3=df2.drop(columns=['Longueur', 'Moyen de transport','Altitude','Commune'])
df3.head()


final_df=df3.assign(Pays="Suisse")
final_df.head()
final_df.to_csv(r'stations.csv', index = False)

# 5.2. Lines

I use a geojson you can find on https://data.sbb.ch/explore/dataset/linie-mit-polygon/export/?fbclid=IwAR3vTCN6GkY4UXZRrm4RNjTRIn726lOGLZmni_K_bi5s-XjerqQ9eCemsrk

In [ ]:
import folium
print(folium.__version__)

lines_suisse=gpd.read_file('linie-mit-polygon.geojson')




We are left with an id for the line, the name of the line, the geographical region in which the line lies (between 'NODEFROM' and 'NODETO'), and the geometry of the lines.

We can't get rid of any further rows, as each row contains unique geometric information